## Data Cleanup
The dataset consists of 10 csv files and each one containes a benign and malicious network traffic collected in different days. Before we start the analysis of data we have to do some data cleaning steps to make sure that our data has no issues and we can use it to train several models.

In the begining we will read one file to have a general look on the data and to know if it has problems. Then we will fix these problems and apply the steps on all files.

In [1]:
# set base path to the directory containing the csv files of the dataset
dataset_base_path = r'dataset'

### 1. Remove the Rows Containing the Headers

In [2]:
import numpy as np
import pandas as pd
import os

file_path = os.path.join(dataset_base_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')

df = pd.read_csv(file_path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


The warning shows that the data type in all columns is not specified. Let's see more information of the columns.

In [3]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331125 entries, 0 to 331124
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Dst Port           331125 non-null  object
 1   Protocol           331125 non-null  object
 2   Timestamp          331125 non-null  object
 3   Flow Duration      331125 non-null  object
 4   Tot Fwd Pkts       331125 non-null  object
 5   Tot Bwd Pkts       331125 non-null  object
 6   TotLen Fwd Pkts    331125 non-null  object
 7   TotLen Bwd Pkts    331125 non-null  object
 8   Fwd Pkt Len Max    331125 non-null  object
 9   Fwd Pkt Len Min    331125 non-null  object
 10  Fwd Pkt Len Mean   331125 non-null  object
 11  Fwd Pkt Len Std    331125 non-null  object
 12  Bwd Pkt Len Max    331125 non-null  object
 13  Bwd Pkt Len Min    331125 non-null  object
 14  Bwd Pkt Len Mean   331125 non-null  object
 15  Bwd Pkt Len Std    331125 non-null  object
 16  Flow Byts/s        3

So all the columns have object data type. Therefore, we will start exploring the values inside some columns to know where is the problem.

In [4]:
df['Protocol'].value_counts()

6           170066
17           95674
6            42833
17           15378
0             4596
0             2553
Protocol        25
Name: Protocol, dtype: int64

In [21]:
df['CWE Flag Count'].value_counts()

0                 269473
0                  60697
1                    863
1                     67
CWE Flag Count        25
Name: CWE Flag Count, dtype: int64

As we see the column name appears inside the column with the values, and the issue exists in all columns. Hence, the headers are duplicated multiple times within the file. As a result, we will remove the headers from the entire rows.

In [6]:
df = df[~df['Dst Port'].str.contains('Dst Port', na=False)]

Next, we will export the data into a new file with defining the correct data types. Also, we will make a little changes to the columns names to make them easier to access.

In [7]:
import re

tmp_path = os.path.join(dataset_base_path, 'tmp')

if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

column_name_regex = re.compile(r"\W", re.IGNORECASE)
df.columns = [column_name_regex.sub('_', c.lower()) for c in df.columns]

tmp_file_path = os.path.join(tmp_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter_duplicate_headers_removed.csv')

df.to_csv(tmp_file_path)

### 2. Replacing invalid values with Correct Ones

Now the temporary file will be loaded with the following datatype definitions. 

In [8]:
types = {
    'dst_port': 'uint32',
    'protocol': 'uint8',
    'timestamp': 'object',
    'flow_duration': 'int64',
    'tot_fwd_pkts': 'uint32',
    'tot_bwd_pkts': 'uint32',
    'totlen_fwd_pkts': 'uint32',
    'totlen_bwd_pkts': 'uint32',
    'fwd_pkt_len_max': 'uint16',
    'fwd_pkt_len_min': 'uint16',
    'fwd_pkt_len_mean': 'float32',
    'fwd_pkt_len_std': 'float32',
    'bwd_pkt_len_max': 'uint16',
    'bwd_pkt_len_min': 'uint16',
    'bwd_pkt_len_mean': 'float32',
    'bwd_pkt_len_std': 'float32',
    'flow_byts_s': 'float64',
    'flow_pkts_s': 'float64',
    'flow_iat_mean': 'float32',
    'flow_iat_std': 'float32',
    'flow_iat_max': 'int64',
    'flow_iat_min': 'int64',
    'fwd_iat_tot': 'int64',
    'fwd_iat_mean': 'float32',
    'fwd_iat_std': 'float32',
    'fwd_iat_max': 'int64',
    'fwd_iat_min': 'int64',
    'bwd_iat_tot': 'uint32',
    'bwd_iat_mean': 'float32',
    'bwd_iat_std': 'float32',
    'bwd_iat_max': 'uint32',
    'bwd_iat_min': 'uint32',
    'fwd_psh_flags': 'uint8',
    'bwd_psh_flags': 'uint8',
    'fwd_urg_flags': 'uint8',
    'bwd_urg_flags': 'uint8',
    'fwd_header_len': 'uint32',
    'bwd_header_len': 'uint32',
    'flow_byts_s': 'float32',
    'bwd_pkts_s': 'float32',
    'pkt_len_min': 'uint16',
    'pkt_len_max': 'uint16',
    'pkt_len_mean': 'float32',
    'pkt_len_std': 'float32',
    'pkt_len_var': 'float32',
    'fin_flag_cnt': 'uint8',
    'syn_flag_cnt': 'uint8',
    'rst_flag_cnt': 'uint8',
    'psh_flag_cnt': 'uint8',
    'ack_flag_cnt': 'uint8',
    'urg_flag_cnt': 'uint8',
    'cwe_flag_count': 'uint8',
    'ece_flag_cnt': 'uint8',
    'down_up_ratio': 'uint16',
    'pkt_size_avg': 'float32',
    'fwd_seg_size_avg': 'float32',
    'bwd_seg_size_avg': 'float32',
    'fwd_byts_b_avg': 'uint8',
    'fwd_pkts_b_avg': 'uint8',
    'fwd_blk_rate_avg': 'uint8',
    'bwd_byts_b_avg': 'uint8',
    'bwd_pkts_b_avg': 'uint8',
    'bwd_blk_rate_avg': 'uint8',
    'subflow_fwd_pkts': 'uint32',
    'subflow_fwd_byts': 'uint32',
    'subflow_bwd_pkts': 'uint32',
    'subflow_bwd_byts': 'uint32',
    'init_fwd_win_byts': 'int32',
    'init_bwd_win_byts': 'int32',
    'fwd_act_data_pkts': 'uint32',
    'fwd_seg_size_min': 'uint8',
    'active_mean': 'float32',
    'active_std': 'float32',
    'active_max': 'uint32',
    'active_min': 'uint32',
    'idle_mean': 'float32',
    'idle_std': 'float32',
    'idle_max': 'uint64',
    'idle_min': 'uint64',
    'label': 'category'
}

In [9]:
df = pd.read_csv(tmp_file_path, dtype=types)

In some file an error related to data type appears because some columns contains a word Infinity but in python it should be either inf or -inf, and so we will replace it and then read the file again.

In [10]:
df = pd.read_csv(tmp_file_path)

df_infinity_fixed = df.replace('Infinity', 'inf')

tmp_file_path_inf = os.path.join(tmp_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter_infinity_fixed.csv')

df_infinity_fixed.to_csv(tmp_file_path_inf)

After fixing the infinity values the file can successfully be loaded with the given datatypes.

In [11]:
df = pd.read_csv(tmp_file_path_inf, dtype=types)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331100 entries, 0 to 331099
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Unnamed: 0         331100 non-null  int64   
 1   Unnamed: 0.1       331100 non-null  int64   
 2   dst_port           331100 non-null  uint32  
 3   protocol           331100 non-null  uint8   
 4   timestamp          331100 non-null  object  
 5   flow_duration      331100 non-null  int64   
 6   tot_fwd_pkts       331100 non-null  uint32  
 7   tot_bwd_pkts       331100 non-null  uint32  
 8   totlen_fwd_pkts    331100 non-null  uint32  
 9   totlen_bwd_pkts    331100 non-null  uint32  
 10  fwd_pkt_len_max    331100 non-null  uint16  
 11  fwd_pkt_len_min    331100 non-null  uint16  
 12  fwd_pkt_len_mean   331100 non-null  float32 
 13  fwd_pkt_len_std    331100 non-null  float32 
 14  bwd_pkt_len_max    331100 non-null  uint16  
 15  bwd_pkt_len_min    331100 non-null

In [13]:
print(f"Infinity values of flow_byts_s: {df[df['flow_byts_s'] == np.inf]['dst_port'].count()}")
print(f"Null values of flow_byts_s: {df[df['flow_byts_s'].isnull()]['dst_port'].count()}")

Infinity values of flow_byts_s: 1085
Null values of flow_byts_s: 1834


### 3. Cleanup Other Files in the Same Way

Here, we will start cleaning all the files and save them in a new directory. We will do the following steps:
1. Remove the duplicated headers.
2. Replace `Infinity` with `inf`.
3. Renaming the column names to remove whitespaces and non-word characters.

We will name each file with the type of attack contained in that file. Also, the columns (`Flow ID`, `Src IP`, `Dst IP`, `Src Port`) in the file `Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv` will be removed because they do not appear in the other files.

In [14]:
import numpy as np
import pandas as pd
import os
import re

csv_files = {
 'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv': 'infiltration_28-02-2018.csv',
 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv': 'infiltration_01-03-2018.csv',
 'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv': 'bot_02-03-2018.csv',
 'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-web-xss_sql-injection_22-02-2018.csv',
 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv': 'dos-goldeneye-slowloris_15-02-2018.csv',
 'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv': 'ddos-loic-http-loic-udp_20-02-2018.csv',
 'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv': 'ddos-loic-udp_hoic_21-02-2018.csv',
 'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-ftp-ssh_14-02-2018.csv',
 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv': 'dos-slowhttp-hulk_16-02-2018.csv',
 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-web-xss_sql-injection_23-02-2018.csv'
}

column_name_regex = re.compile(r"\W", re.IGNORECASE)
processed_dir = 'processed'
processed_path = os.path.join(dataset_base_path, processed_dir)

def remove_headers(f):    
    return f[~f['Dst Port'].str.contains('Dst Port', na=False)]

def replace_infinity(f):
    return f.replace('Infinity', 'inf', inplace=True)

def remove_non_word_chars_from_column_names(f):
    return [column_name_regex.sub('_', c.lower()) for c in df.columns]
    
if not os.path.exists(processed_path):
    os.mkdir(processed_path)    
    
for f, out in csv_files.items():
    file_path = os.path.join(dataset_base_path, f)
    output_path = os.path.join(dataset_base_path, processed_dir, out)
    
    df = pd.read_csv(file_path, dtype=str).drop(columns=['Flow ID', 'Src IP', 'Dst IP', 'Src Port'], errors='ignore')
    df = remove_headers(df)
    replace_infinity(df)
    df.columns = remove_non_word_chars_from_column_names(df)
    df.to_csv(output_path, index=False)

In [2]:
import numpy as np
import pandas as pd
df2=pd.read_csv('dataset/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')

In [5]:
df3=df2[['Flow ID', 'Src IP', 'Dst IP', 'Src Port']]

In [10]:
df3.head()

,Flow ID,Src IP,Dst IP,Src Port
0,172.31.69.25-94.231.103.172-22-45498-6,94.231.103.172,172.31.69.25,45498
1,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,8.0.6.4,0
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,8.0.6.4,0
3,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,8.0.6.4,0
4,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,8.0.6.4,0


In [16]:
df3.shape

(7948748, 4)

In [12]:
df3['Flow ID'].value_counts()

8.0.6.4-8.6.0.1-0-0-0                       93775
172.31.67.120-13.89.188.5-49674-443-6         350
172.31.67.38-13.89.187.212-49674-443-6        348
172.31.66.78-13.89.184.238-49672-443-6        339
172.31.66.95-13.89.188.5-49672-443-6          331
                                            ...  
144.21.81.16-172.31.64.71-60905-3389-6          1
172.31.65.57-175.195.219.31-3389-60440-6        1
172.31.0.2-172.31.65.29-53-57774-17             1
172.31.0.2-172.31.67.59-53-57210-17             1
172.31.67.21-35.167.62.142-3389-56391-6         1
Name: Flow ID, Length: 5030830, dtype: int64

In [13]:
df3['Src IP'].value_counts()

8.6.0.1           93775
18.219.9.1        58532
18.218.229.235    58398
52.14.136.135     58137
18.218.55.126     57998
                  ...  
27.18.128.146         1
94.25.168.185         1
52.85.133.124         1
223.67.233.173        1
95.134.48.11          1
Name: Src IP, Length: 31291, dtype: int64

In [14]:
df3['Dst IP'].value_counts()

172.31.0.2         2457307
172.31.69.25        576782
169.254.169.254     549148
8.0.6.4              93775
178.255.83.1         45386
                    ...   
54.231.168.251           1
185.13.229.59            1
140.205.34.51            1
111.250.56.54            1
52.216.227.235           1
Name: Dst IP, Length: 27076, dtype: int64

In [15]:
df3['Src Port'].value_counts()

443     630995
445     210031
80      153777
0       144134
3389     80222
         ...  
5310         1
485          1
33           1
483          1
893          1
Name: Src Port, Length: 64898, dtype: int64